In [1]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/28 22:24:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

In [2]:


# lista wartości zostaje podzielona na partycje i rozproszona na wszystkie dostępne węzły
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.SparkContext.parallelize.html
rdd = spark.sparkContext.parallelize(list(range(20)))

# rdd w formie rozproszonej zostaje scalone w listę zawierającą wszystkie elementy RDD
# np. za pomocą funkcji collect()
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.RDD.collect.html

rddCollect = rdd.collect()
display(type(rddCollect))
print(f"Liczba partycji: {rdd.getNumPartitions()}")
print(f"Pierwszy element: {rdd.first()}")
print(rddCollect)



list

Liczba partycji: 2
Pierwszy element: 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [5]:
# obiekt RDD może przechowywać dane z różnych źródeł, które są zgodne z systemem plików Apache Hadoop
# np. Amazon S3, Cassandra, HDFS, HBase i inne

# możemy dla uniknięcia potrzeby każdorazowego odwoływania się do kontekstu poprzez spark.sparkContext zapisać sobie to w zmiennej pomocniczej
sc = spark.sparkContext
# tutaj wczytamy do RDD plik tekstowy
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

In [6]:
display(pan_tadeusz_file.getNumPartitions())
pan_tadeusz_file.take(10)

2

['Adam Mickiewicz',
 '',
 'Pan Tadeusz',
 'czyli ostatni zajazd na Litwie',
 '',
 'ISBN 978-83-288-2495-9',
 '',
 '',
 '',
 '']

In [7]:
# możemy zmienić liczbę automatycznie stworzonych partycji i ponownie rozproszyć je po węzłach
pan_tadeusz_file = pan_tadeusz_file.repartition(4)
pan_tadeusz_file.getNumPartitions()

# również metoda coalesce może posłużyć nam do zmiany ilości partycji dla obiektu RDD np. po zastosowaniu filtrowania, które
# znacznie zmniejsza wielkość pierwotnego obiektu RDD a co za tym idzie każdej partycji i dalsze obliczenia mogą nie być
# wykonywane zbyt efektywnie (zbyt mały rozmiar partycji)
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.RDD.coalesce.html
# główna różnica między repartition a coalesce jest taka, że ta pierwsza wykorzystuje mechanizm tasowania danych a ta druga może, ale nie
# musi go wykorzystywać gdyż możemy tym sterować za pomocą parametru wywołania tej metody


4

In [8]:
# jedną z funkcji dostępnej w tym API jest możliwość wykonania funkcji na każdej z partycji
# minusem może być to, że funkcja foreachPartition zwraca typ None, więc wyniki należy przetworzyć w inny sposób

def count_words(iterator):
    words = sum([len(x.split()) for x in iterator])
    print(words)

pan_tadeusz_file.foreachPartition(count_words)

17366
17129
17307
17293


In [9]:


# funkcje map oraz reduce

# możemy również wykonać operację w inny sposób, tym raze mapując funkcję na każdy element obiektu RDD
# zwrócony zostanie obiekt RDD, na którym możemy wykonać kolejne operacje
pan_tadeusz_file.map(lambda s: len(s.split())).take(10)

# np. reduce - i tu nawiązanie do dość znanej techniki przetwarzania, MapReduce
# więcej: https://en.wikipedia.org/wiki/MapReduce
# oraz: https://wiadrodanych.pl/big-data/jak-dziala-mapreduce/
pan_tadeusz_file.map(lambda s: len(s.split())).reduce(lambda a, b: a + b)



69095

In [10]:
# lub tak - ten sam efekt
from operator import add
pan_tadeusz_file.map(lambda s: len(s.split())).reduce(add)

69095

In [11]:
# różnica między map() a flatMap() dla tego przypadku
display(pan_tadeusz_file.map(lambda s: s.split()).take(10))
pan_tadeusz_file.flatMap(lambda s: s.split()).take(10)

[['I', 'zaraz', 'mogłem', 'pieszo,', 'do', 'Twych', 'świątyń', 'progu'],
 ['Iść', 'za', 'wrócone', 'życie', 'podziękować', 'Bogu),'],
 ['Tak', 'nas', 'powrócisz', 'cudem', 'na', 'Ojczyzny', 'łono.'],
 ['Tymczasem', 'przenoś', 'moją', 'duszę', 'utęsknioną'],
 ['Do', 'tych', 'pagórków', 'leśnych,', 'do', 'tych', 'łąk', 'zielonych,'],
 ['Szeroko', 'nad', 'błękitnym', 'Niemnem', 'rozciągnionych;'],
 ['Do', 'tych', 'pól', 'malowanych', 'zbożem', 'rozmaitem,'],
 ['Wyzłacanych', 'pszenicą,', 'posrebrzanych', 'żytem;'],
 ['Gdzie', 'bursztynowy', 'świerzop,', 'gryka', 'jak', 'śnieg', 'biała,'],
 ['Gdzie', 'panieńskim', 'rumieńcem', 'dzięcielina', 'pała,']]

['I',
 'zaraz',
 'mogłem',
 'pieszo,',
 'do',
 'Twych',
 'świątyń',
 'progu',
 'Iść',
 'za']

In [12]:
tadeusz_lines = pan_tadeusz_file.filter(lambda line: "Tadeusz" in line)
tadeusz_count = tadeusz_lines.count()
print(f"Liczba linii zawierających słowo 'Tadeusz': {tadeusz_count}")

Liczba linii zawierających słowo 'Tadeusz': 182


In [15]:
longest_lines = pan_tadeusz_file.map(lambda line: (len(line), line)).top(3, key=lambda x: x[0])
print("Trzy najdłuższe linie:")
for length, line in longest_lines:
    print(f"Długość: {length}, Treść: {line}\n\n")

Trzy najdłuższe linie:
Długość: 536, Treść: O niebezpieczeństwach wynikających z nieporządnego obozowania — Odsiecz niespodziana — Smutne położenie szlachty — Odwiedziny kwestarskie są wróżbą ratunku — Major Płut zbytnią zalotnością ściąga na siebie burzę — Wystrzał z krócicy, hasło boju — Czyny Kropiciela, czyny i niebezpieczeństwa Maćka — Konewka zasadzką ocala Soplicowo — Posiłki jezdne, atak na piechotę — Czyny Tadeusza — Pojedynek dowódców przerwany zdradą — Wojski stanowczym manewrem przechyla szalę boju — Czyny krwawe Gerwazego — Podkomorzy zwycięzca wspaniałomyślny.


Długość: 452, Treść: Plany myśliwskie Telimeny — Ogrodniczka wybiera się na wielki świat i słucha nauk opiekunki — Strzelcy wracają — Wielkie zadziwienie Tadeusza — Spotkanie się powtórne w Świątyni dumania i zgoda ułatwiona za pośrednictwem mrówek — U stołu wytacza się rzecz o łowach — Powieść Wojskiego o Rejtanie i księciu Denassów, przerwana — Zagajenie układów między stronami, także przerwane — Zjawisko z kluc

In [18]:
stopwords = sc.textFile('polish.stopwords.txt').collect()

In [20]:
# Funkcja czyszcząca tekst (usuwanie znaków przestankowych i zmiana na małe litery)
def clean_text(line):
    line = re.sub(r'[^\w\s]', '', line)  # Usuń znaki przestankowe
    line = line.lower()                 # Zamień na małe litery
    return line.split()

# Wyczyść dane i odfiltruj stopwords
words = pan_tadeusz_file.flatMap(clean_text).filter(lambda word: word not in stopwords)

# Policz wystąpienia każdego słowa
word_counts = words.countByValue()

# Zapisz do pliku JSON
output_file = "pan_tadeusz_bag_of_words.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(word_counts, f, ensure_ascii=False, indent=4)

# Znajdź najczęściej występujące słowo
most_common_word = max(word_counts.items(), key=lambda x: x[1])
print(f"Najczęściej występujące słowo: {most_common_word[0]}, Liczba wystąpień: {most_common_word[1]}")

25/01/28 22:33:04 ERROR Executor: Exception in task 1.0 in stage 26.0 (TID 42)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/work-dir/venv/lib/python3.10/site-packages/pyspark/rdd.py", line 1919, in func
    initial = next(iterator)
  File "/opt/spark/work-dir/venv/lib/python3.10/site-packages/pyspark/rdd.py", line 2653, in countPartition
    for obj in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_589/3400670931.py", line 3, in clean_text
NameError: name 're' is not defined

	at

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 26.0 failed 1 times, most recent failure: Lost task 1.0 in stage 26.0 (TID 42) (a5f5bf954455 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/work-dir/venv/lib/python3.10/site-packages/pyspark/rdd.py", line 1919, in func
    initial = next(iterator)
  File "/opt/spark/work-dir/venv/lib/python3.10/site-packages/pyspark/rdd.py", line 2653, in countPartition
    for obj in iter